<a href="https://colab.research.google.com/github/FernandoBRdgz/diplomado_ds_mod4/blob/main/4.6%20Modelaci%C3%B3n%20supervisada%20y%20no%20supervisada%20de%20Big%20Data/4.6.1%20Spark%20MLlib.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Ejemplo Introductorio

In [ ]:
generation = "mod4gen<x>"

In [ ]:
data = spark.read.format("libsvm").load(f"/mnt/{generation}/input/sample_linear_regression_data.txt", inferSchema=True, header=True)

In [ ]:
data.count(), data.columns

Es común en la práctica tener datos de entrenamiento poco densos. MLlib admite la lectura de registros de entrenamiento almacenados en formato `LIBSVM`, que es un formato de texto predeterminado en el que cada línea representa un vector de características dispersas etiquetadas.

In [ ]:
data.display()

In [ ]:
data.show()

In [ ]:
type(data)

In [ ]:
# Train/val split
train, val = data.randomSplit([0.8, 0.2])

In [ ]:
train.count(), val.count()

In [ ]:
train.show()

In [ ]:
val.show()

In [ ]:
unlabeled_val = val.select("features")

In [ ]:
unlabeled_val.show()

El formato que espera Spark MLlib son dos columnas, una con la variable objetivo y otra con el listado de variables.

La columna asociada a la variable objetivo debe ser de tipo numérico, ya sea un valor continuo para regresión o un valor discreto que coincida con una categoría de clasificación. Más adelante hablaremos de algoritmos de aprendizaje no supervisado que por su naturaleza no utilizan ni requieren etiquetas.

La columna de características tiene dentro un vector con todas las variables que pertenecen a esa fila. Por lo general, lo que se suele hacer es combinar las diversas variables con las que contamos en una sola columna de características.

In [ ]:
from pyspark.ml.regression import LinearRegression

In [ ]:
help(LinearRegression)

In [ ]:
# Estos son los valores predeterminados para 'featuresCol', 'labelCol' y 'predictionCol'
lr = LinearRegression(featuresCol="features", labelCol="label", predictionCol="prediction")

In [ ]:
# Entrenamiento de modelo
lr_model = lr.fit(train)

In [ ]:
# Imprime los coeficientes y el intercepto para la regresión lineal
print("Coefficientes: {}".format(str(lr_model.coefficients)))
print("Intercepto:{}".format(str(lr_model.intercept)))

In [ ]:
# Resumen del modelo
train_results = lr_model.summary

In [ ]:
type(train_results)

In [ ]:
train_results.residuals.display()

In [ ]:
# Predicciones
predictions = lr_model.transform(unlabeled_val)

In [ ]:
predictions.show()

In [ ]:
val_results = lr_model.evaluate(val)

In [ ]:
# Evaluación train
print("RMSE: {}".format(train_results.rootMeanSquaredError))
print("r2: {}".format(train_results.r2))

In [ ]:
# Evaluación test
print("RMSE: {}".format(val_results.rootMeanSquaredError))
print("r2: {}".format(val_results.r2))